In [8]:
import keras
import numpy as np
import pandas as pd
from time import time
#from utils import dmanip, vis
from keras.models import Sequential
from keras.layers import LSTM, Dense
#from keras.utils import to_categorical
from keras.utils.np_utils import to_categorical
from keras.callbacks import TensorBoard
from sklearn.preprocessing import LabelEncoder
from tensorflow.python.client import device_lib
from sklearn.model_selection import train_test_split

In [9]:
# get data
data = pd.read_csv('Level_3b_NonPath_SP2_EsCo_PSL_1_2_3_4_574.csv') 

# split data
X=data.iloc[:,1:151]
y=data.iloc[:,151].astype('category')

print(X)
print(y)

         Xc1.A     Xc1.R  ...  Grantham.Xd.29  Grantham.Xd.30
0     2.566136  2.332851  ...        0.032370        0.030361
1    17.798460  6.396322  ...        0.032770        0.032138
2     5.387067  2.835298  ...        0.031862        0.027646
3    13.095425  1.636928  ...        0.030777        0.032884
4     9.634552  5.831440  ...        0.032222        0.031622
..         ...       ...  ...             ...             ...
569  16.638587  8.839250  ...        0.035352        0.034787
570  13.692819  4.471124  ...        0.031048        0.031582
571   4.115225  3.566528  ...        0.029097        0.031874
572  12.470586  6.495097  ...        0.030560        0.031774
573   4.793531  1.597844  ...        0.031451        0.031101

[574 rows x 150 columns]
0      1
1      1
2      1
3      1
4      1
      ..
569    4
570    4
571    4
572    4
573    4
Name: Class, Length: 574, dtype: category
Categories (4, int64): [1, 2, 3, 4]


In [10]:
# init
n_epochs = 100
n_comps = X.shape[1]

In [11]:
# encode the classification labels
le = LabelEncoder()
yy = to_categorical(le.fit_transform(y))

from collections import Counter
from imblearn.over_sampling import SMOTE 


sm = SMOTE(random_state=45)
x_b, y_b = sm.fit_resample(X, yy)


# split the dataset
x_train, x_test, y_train, y_test = train_test_split(x_b, y_b, test_size=0.1,
                                                    random_state=True,
                                                    shuffle=True)

# exapand dimensions
x_train = np.expand_dims(x_train, axis=2)
x_test = np.expand_dims(x_test, axis=2)
#y_train = np.expand_dims(y_train, axis=2)
#y_test = np.expand_dims(y_test, axis=2)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [12]:
# define model
model = Sequential()
model.add(LSTM(units=n_comps, return_sequences=True,
               input_shape=(x_train.shape[1], 1),
               dropout=0.2, recurrent_dropout=0.2))
model.add(LSTM(128, return_sequences=True, dropout=0.2, recurrent_dropout=0.2))
model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))
#model.add(Dense(128, input_dim = 90, activation = 'relu'))
#model.add(Dense(64, activation = 'relu'))
model.add(Dense(4 ,activation='softmax'))

# print model architecture summary
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 150, 150)          91200     
_________________________________________________________________
lstm_4 (LSTM)                (None, 150, 128)          142848    
_________________________________________________________________
lstm_5 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 260       
Total params: 283,716
Trainable params: 283,716
Non-trainable params: 0
_________________________________________________________________
None


In [13]:
# compile model
#model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
history = model.fit(x_train, y_train, epochs=n_epochs, batch_size=64,
                    validation_data=(x_test, y_test))

Epoch 1/100
17/17 [==============================] - 40s 2s/step - loss: 1.3877 - accuracy: 0.2442 - val_loss: 1.3893 - val_accuracy: 0.2034
Epoch 2/100
17/17 [==============================] - 23s 1s/step - loss: 1.3870 - accuracy: 0.2624 - val_loss: 1.3890 - val_accuracy: 0.2034
Epoch 3/100
17/17 [==============================] - 24s 1s/step - loss: 1.3865 - accuracy: 0.2657 - val_loss: 1.3889 - val_accuracy: 0.2203
Epoch 4/100
17/17 [==============================] - 24s 1s/step - loss: 1.3858 - accuracy: 0.2544 - val_loss: 1.3881 - val_accuracy: 0.2034
Epoch 5/100
17/17 [==============================] - 24s 1s/step - loss: 1.3866 - accuracy: 0.2296 - val_loss: 1.3879 - val_accuracy: 0.2034
Epoch 6/100
17/17 [==============================] - 24s 1s/step - loss: 1.3861 - accuracy: 0.2547 - val_loss: 1.3886 - val_accuracy: 0.2203
Epoch 7/100
17/17 [==============================] - 24s 1s/step - loss: 1.3862 - accuracy: 0.2475 - val_loss: 1.3890 - val_accuracy: 0.2034
Epoch 8/100
1

In [22]:
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
#from sklearn.metrics import roc_auc_score
import itertools

train_pred = model.predict(x_train)
test_pred = model.predict(x_test)
print("train-acc = " + str(accuracy_score(np.argmax(y_train, axis=1), np.argmax(train_pred, axis=1))))
print("test-acc = " + str(accuracy_score(np.argmax(y_test, axis=1), np.argmax(test_pred, axis=1))))
print("train-CFR = " + str(classification_report(np.argmax(y_train, axis=1), np.argmax(train_pred, axis=1))))
print("test-CFR = " + str(classification_report(np.argmax(y_test, axis=1), np.argmax(test_pred, axis=1))))
#print((roc_auc_score(np.argmax(y_train, axis=1), np.argmax(train_pred, axis=1))))

train-acc = 0.2551984877126654
test-acc = 0.2033898305084746
train-CFR =               precision    recall  f1-score   support

           0       0.00      0.00      0.00       260
           1       0.26      1.00      0.41       270
           2       0.00      0.00      0.00       260
           3       0.00      0.00      0.00       268

    accuracy                           0.26      1058
   macro avg       0.06      0.25      0.10      1058
weighted avg       0.07      0.26      0.10      1058

test-CFR =               precision    recall  f1-score   support

           0       0.00      0.00      0.00        34
           1       0.20      1.00      0.34        24
           2       0.00      0.00      0.00        34
           3       0.00      0.00      0.00        26

    accuracy                           0.20       118
   macro avg       0.05      0.25      0.08       118
weighted avg       0.04      0.20      0.07       118



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
